In [13]:
from pwn import *

In [14]:
# Mino mask defs: toszlji
masks = [
    (0, 0, (2, 3), [0b010, 0b111]),
    (0, 1, (3, 2), [0b10, 0b11, 0b10]),
    (0, 2, (2, 3), [0b111, 0b010]),
    (0, 3, (3, 2), [0b01, 0b11, 0b01]),

    (1, 0, (2, 2), [0b11, 0b11]),
    (1, 1, (2, 2), [0b11, 0b11]),
    (1, 2, (2, 2), [0b11, 0b11]),
    (1, 3, (2, 2), [0b11, 0b11]),

    (2, 0, (2, 3), [0b011, 0b110]),
    (2, 1, (3, 2), [0b10, 0b11, 0b01]),
    (2, 2, (2, 3), [0b011, 0b110]),
    (2, 3, (3, 2), [0b10, 0b11, 0b01]),

    (3, 0, (2, 3), [0b110, 0b011]),
    (3, 1, (3, 2), [0b01, 0b11, 0b10]),
    (3, 2, (2, 3), [0b110, 0b011]),
    (3, 3, (3, 2), [0b01, 0b11, 0b10]),

    (4, 0, (3, 2), [0b10, 0b10, 0b11]),
    (4, 1, (2, 3), [0b111, 0b100]),
    (4, 2, (3, 2), [0b11, 0b01, 0b01]),
    (4, 3, (2, 3), [0b001, 0b111]),

    (5, 0, (3, 2), [0b01, 0b01, 0b11]),
    (5, 1, (2, 3), [0b100, 0b111]),
    (5, 2, (3, 2), [0b11, 0b10, 0b10]),
    (5, 3, (2, 3), [0b111, 0b001]),

    (6, 0, (4, 1), [0b1, 0b1, 0b1, 0b1]),
    (6, 1, (1, 4), [0b1111]),
    (6, 2, (4, 1), [0b1, 0b1, 0b1, 0b1]),
    (6, 3, (1, 4), [0b1111])
]

In [50]:
board = [
    0b1111000010100000,
    0b0000110111011100,
    0b1110111101110111,
    0b0010101011111010,
    0b0001011011001010,
    0b1111010011001110,
    0b0111011011110100,
    0b0100011010010110,
    0b1101110111110100,
    0b0110101000010010,
    0b1000110001101111,
    0b1111111111111111
]
board_w = 16
board_h = len(board)

confirmed_mask = [0 for _ in range(board_w)]

In [60]:
def detect_free_br_corner():
    # Detect bottom right corner
    area_b = -1
    for y in range(board_h-1, 0, -1):
        if board[y] != 0:
            area_b = y
            break
    print("area_b", area_b)

    for x in range(board_w-1, 0, -1):
        if (board[area_b] &~ confirmed_mask[area_b]) & (1 << x):
            area_r = x
            break

    return area_b, area_r


In [80]:
confirmed_mask = [0 for _ in range(board_w)]

In [81]:
area_b, area_r = detect_free_br_corner()
print(f"area_b: {area_b}, area_r: {area_r}")

for (mino_type, mino_rotation, mino_area, mask) in masks:
    area_h = mino_area[0]
    area_w = mino_area[1]

    area_x = area_r - area_w + 1
    area_y = area_b - area_h + 1
    print(f"{mino_type}/{mino_rotation} area_x: {area_x}, area_y: {area_y}")

    area_shift_row = board_w - area_r + 1
    area_mask_row = (1 << area_w) - 1


    area = [
        (board[area_y + i] & ~confirmed_mask[area_y + i] ) >> area_shift_row & area_mask_row
        for i in range(area_h)
    ]
    area_match = all(
        area[i] & mask[i] == mask[i]
        for i in range(area_h)
    )
    if not area_match:
        continue

    if (mino_type, mino_rotation) == (0, -1):
        print("Skip 0/0")
        continue
    # Confirm mask
    for i in range(area_h):
        confirmed_mask[area_y + i] |= mask[i] << area_shift_row


    print(f"mino: {mino_type}/{mino_rotation}, h,w: {mino_area}, d:{area_shift_row}${area_mask_row}, {mask} & {area} -> {area_match}, area on {area_y}, {area_x}")
    break

area_b 11
area_b: 11, area_r: 15
0/0 area_x: 13, area_y: 10
mino: 0/0, h,w: (2, 3), d:2$7, [2, 7] & [3, 7] -> True, area on 10, 13


In [82]:
corner = detect_free_br_corner()
print(f"corner: {corner}")
print(f"board: {board_h}x{board_w}")

for i in range(board_h):
    line = ""
    if i == area_b:
        line += "\x1b[31m"
    line += f"{i:02} "
    if i == area_b:
        line += "\x1b[0m"

    for j in range(board_w):
        mask = (1 << (board_w - j - 1))
        modes = []

        if confirmed_mask[i] & mask:
            modes.append("42")
        elif board[i] & mask:
            ch = "#"
        else:
            ch = "."

        if i == area_b and j == area_r:
            modes.append("31")

        modes_txt = ";".join(modes)
        line += f"\x1b[0;{modes_txt}m{ch}\x1b[0m"
    print(line)

area_b 11
corner: (11, 15)
board: 12x16
00 ####....#.#.....
01 ....##.###.###..
02 ###.####.###.###
03 ..#.#.#.#####.#.
04 ...#.##.##..#.#.
05 ####.#..##..###.
06 .###.##.####.#..
07 .#...##.#..#.##.
08 ##.###.#####.#..
09 .##.#.#....#..#.
10 #...##...##..###
11 ################
